# LangGraph + MCP (Notebook-safe) + Ollama Autonomous Agent

**PLAN → Human Approval → ACT**

- Notebook-safe MCP tools
- No ClientSession
- Fully autonomous & expandable


In [24]:
!pip install -U langgraph langchain langchain-ollama mcp

In [25]:
from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph, END
from langchain_ollama import ChatOllama
import json
import os

In [26]:
llm = ChatOllama(
    model="gpt-oss:20b",
    temperature=0
)

## Agent State

In [27]:
class AgentState(TypedDict):
    goal: str
    plan: List[str]
    approved: bool
    messages: List[Dict]
    done: bool

## MCP Tools (In-process, Notebook-safe)

In [28]:
from mcp.server.fastmcp import FastMCP

BASE_DIR = "/tmp/mcp_workspace"
os.makedirs(BASE_DIR, exist_ok=True)

mcp_server = FastMCP("filesystem")

@mcp_server.tool()
def list_dir(path: str = "."):
    return os.listdir(os.path.join(BASE_DIR, path))

@mcp_server.tool()
def read_file(path: str) -> str:
    return open(os.path.join(BASE_DIR, path)).read()

@mcp_server.tool()
def write_file(path: str, content: str) -> str:
    with open(os.path.join(BASE_DIR, path), "w") as f:
        f.write(content)
    return "ok"

## MCP Tool Access Helpers

In [29]:
def list_tools():
    return list(mcp_server._tools.keys())

def call_tool(tool_name: str, args: dict):
    tool = mcp_server._tools[tool_name]
    return tool(**args)

## Planner Node (PLAN MODE)

In [36]:
def planner_node(state: AgentState) -> AgentState:
    if state["plan"]:
        return state
    prompt = f"""
    You are an AI planner.

    Goal:
    {state['goal']}

    Create a step-by-step plan.
    Do NOT execute anything.
    """

    plan_text = llm.invoke(prompt).content

    state['plan'] = [l for l in plan_text.splitlines() if l.strip()]
    state['approved'] = False
    state['messages'] = []
    state['done'] = False
    return state

## Executor Node (Autonomous Tool Choice)

In [37]:
def executor_node(state: AgentState) -> AgentState:
    tools = list_tools()

    prompt = f"""
    You are an autonomous agent.

    Goal: {state['goal']}
    Plan: {state['plan']}
    History: {state['messages']}

    Available tools:
    {tools}

    Decide ONE next action.

    Respond ONLY in JSON:
    {{
      "tool": "<tool_name | done>",
      "args": {{}}
    }}
    """

    action = json.loads(llm.invoke(prompt).content)

    if action['tool'] == 'done':
        state['done'] = True
        return state

    state['messages'].append(action)
    return state

## Tool Runner Node

In [38]:
def tool_node(state: AgentState) -> AgentState:
    action = state['messages'][-1]

    result = call_tool(action['tool'], action.get('args', {}))

    state['messages'].append({'observation': result})
    return state

## Build LangGraph

In [39]:
graph = StateGraph(AgentState)

graph.add_node('planner', planner_node)
graph.add_node('executor', executor_node)
graph.add_node('tool', tool_node)

graph.set_entry_point('planner')
graph.add_conditional_edges(
    'planner',
    lambda s: 'executor' if s['approved'] else 'planner'
)

graph.add_conditional_edges(
    'executor',
    lambda s: END if s['done'] else 'tool'
)

graph.add_edge('tool', 'executor')

app = graph.compile()

## Run PLAN MODE

In [41]:
state = {
    'goal': 'Read docs and write short notes into summary.txt',
    'plan': [],
    'approved': False,
    'messages': [],
    'done': False
}

state = app.invoke(state)
state['plan']

[01/29/26 02:40:52] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=370747;file:///data/praneeth/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=645793;file:///data/praneeth/.venv/lib/python3.13/site-packages/httpx/_client.py#1025\1025]8;;\

GraphRecursionError: Recursion limit of 10000 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/GRAPH_RECURSION_LIMIT

## User clicks ACT

In [ ]:
state['approved'] = True
state = app.invoke(state)
state['messages']